# Forest Stand Height Functions

In [ ]:
def Crop_ISCE:
    xmlfile = "resampleOnlyImage.amp.xml"
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    size_array = np.array([])
    for size in root.iter('property'):
        if size.items()[0][1] == 'size':
            size_array = np.append(size_array, int(size.find('value').text))
    width = size_array[0]
    length = size_array[1]
    
    nanval = 0
    
    # Read amp files in radar coordinates
    amp_file = np.fromfile("resampOnlyImage.amp", dtype = 'complex64')
    inty = amp_file.reshape((length, width))
    
    # Creating empty array for cropped square list
    
    inty[:176,:] = nanval
    inty[5488:,:] = nanval
    inty[:,:163] = nanval
    inty[:,4846:] = nanval
    
    # Write output files
    inty.tofile("resampOnlyImage.amp")

In [ ]:
def Crop_ROIPAC(directory, date1, date2):
    # Extract ROI_PAC parameters
    amp_rsc_file = date1 + "-" + date2 + "-" + "_2rlk.amp.rsc"
    width = int(rrd.read_rsc_data(amp_rsc_file, directory, "WIDTH"))
    length = int(rrd.read_rsc_data(amp_rsc_file, directory, "FILE_LENGTH"))
    fullwidth = width*2
    nanval = 0
    
    # Readcpr files in radar coordinates
    cor_file = np.fromfile(directory + date1 + "-" + date2 + "_2rlks.cor", dtype = "f4", count = length*fullwidth)
    corr = cor_file.reshape((length, fullwidth))
    mag = corr[:, 0:width]
    phs = corr[:.width:fullwidth]
    
    # Read amp files in radar coordinates
    amp_file = np.fromfile(directory + date1 + "-" + date2 + "_2rlks.amp", dtype = 'complex64')
    inty = amp_file.reshape((length, width))
    
    # Creating empty array for cropped square list
    mag[:638, :] = nanval
    mag[3288:, :] = nanval
    mag[:,:84] = nanval
    mag[:,2418:] = nanval
    
    phs[:638, :] = nanval
    phs[3288:, :] = nanval
    phs[:,:84] = nanval
    phs[:,2418:] = nanval
    
    inty[:638, :] = nanval
    inty[3288:, :] = nanval
    inty[:,:84] = nanval
    inty[:,2418:] = nanval
    
    # Writing values
    c_out[:, 0:width] = mag
    c_out[:,width:fullwidth] = phs
    
    # Write output files
    cx = c_out.astype('f4')
    cx.tofile(directory + date1 + "-" + date2 + "_2rlks_fix.cor")
    inty.tofile(directory + date1 + "-" + date2 + "_2rlks_fix.amp")

In [ ]:
def arc_sin(x, C_param):
    # Get rid of extreme values by set all values where x > 1 equal to 1, and x < 0 equal to 0
    x[(x > 1)] = 1
    x[(x < 0)] = 0
    
    # Create array of increments between 0 and pi of size pi/100
    XX = linspace(0, math.pi, num = 100, endpoint = True)
    
    # Set the first value of XX to eps to avoid division by zero issues
    XX[0] = spacing(1)
    
    # Calculate sinc for for XX and save it to YY
    ## YY - sinc(XX / math.pi)
    YY = np.sin(XX) / XX
    
    # Reset the first value of XX to zero and the first value of YY to the corresponding output
    XX[0] = 0
    YY[0] = 1
    
    # Set the last value of YY to 0 to avoid NaN issues
    YY[-1] = 0
    
    # Flip XX and YY left to right
    YY = YY[::-1]
    XX = XX[::-1]
    
    # Run interpolation
    # XX and YY are your original values, x is the query value, and y is the interpolated values that correspond to x
    y = interp_func(x)
    
    # Set all values in y less than 0 equal to 0
    y[(y < 0)] = 0
    # return y
    return y

In [ ]:
# Input parameters are the numbers of scenes, edges, start scene, iterations, the input/output file directory, 
    # averaging numbers in lat and lon for "self" and "pairwise" fitting, bin_size for density calculation in scatter plot fitting, 
    # flag for sparse data cloud filtering.
def auto_mosaicking_new(scenes, edges, start_scene, N, linkarray, directory, Nd_pairwise, Nd_self, bin_size, flag_sparse):
    # Set average S and C parameters (0 < s < 1, 0 < c < 20 so s = 0.65, and c = 13)
    avg_S = 0.65
    avg_C = 13
    
    # Create avg_dp matric, and fill with average S and C parameters
    avg_dp = zeros(scenes * 2)
    put(avg_dp, range(0, scenes * 2, 2), avg_S)
    put(avg_dp, range(1, scenes * 2, 2), avg_C)
    
    # Create the dp matrix
    # the difference of the avg and the initial SC values OR all the zeros (avg)
    dp = zeros(scenes * 2)
    
    # Intialize target matrix nd fill with K = 1, B = 0
    target_KB = zeros((edges + 1) * 2)
    put(target_KB, range(0, (edges + 1) * 2, 2), 1)
    
    # Run cal_KB()
    Y = cKB.cal_KB(dp, edges, start_scene, linkarray, directory, Nd_pairwise, Nd_self, bin_size, flag_sparse)
    
    # Calculate the residual for cal_KB - target
    res = sum((Y - target_KB)**2)
    
    # Save dp and the residuals as the first iteration output file (using JSON)
    iter_file = open(os.path.join(dirextory, "output", "SC_0_iter.json"), 'w')
    json.dump([dp.tolist(), res], iter_file)
    iter_file.close()
    
    # For the rest of the iterations run ls_deltaSC() and save to output file (using JSON)
    for i in range(1, N + 1, 2): # This will run from i = 1 to i = N
        [dp, res] = lSC.ls_deltaSC(dp, edges, scenes, start_scene, linkarray, directory, Nd_pairwise, Nd_self, bin_size, flag_sparse)
        print("%d iterations completed!\n" % i)
        print(time.strftime("%H:%M:%S"))
        filename = "SC_%d_iter.json" % i
        iter_file = open(os.path.join(directory, "output", filename), 'w')
        json.dump([dp.tolist(), res], iter_file)
        iter_file.close()
        
    print("auto_mosaicking_new finsihed at " + (time.strftime("%H:%M:%S")))

In [ ]:
def auto_tree_height_many(scenes, flagfile, directory, numLooks, noiselevel, flag_proc, flag_grad):
    # for each scene name the file, run auto_tree_height_single and save the output to a .json file
    for i in range(scenes):
        
        # Get the scene data and set the file name and image folder name (f#_o# where # is the frame and orbit numbers, respectively)
        scene_data = fsf.flag_scene_file(flagfile, i + 1, directory) # 0 vs1 indexing
        filename = scene_data[1]
        image_folder = "f" + scene_data[4] + "_o" + scene_data[5] + "/"
        
        # Run auto_tree_height_single
        if flag_proc == 0:
            ######## ROI_PAC results
            file_date = athsR.auto_tree_height_single_ROIPAC(directory + image_folder, scene_data[2], scene_data[3], numLooks, noiselevel, flag_grad)
        elif flag_proc == 1:
            ######## ISCE results
            file_data = athsI.auto_tree_height_single_ISCE(directory + image_folder, scene_data[2], scene_data[3], numLooks, noiselevel, flag_grad)
        else:
            print("Invalid processor provided")
            
        linkfile = directory + image_folder + filename + '_orig.mat'
        
    sio.savemat(linkfile, {'corr_vs':file_data[0], 'ks':file_data[1], 'coords':file_data[2]})
    
    # Write geodata to a text file (4th - 9th values in file_data) 
    geofile = open(directory + image_folder + filename + "_geo.txt", "w")
    geofile.write("width: %d \n" % file_data[3])
    geofile.write("nlines: %d \n" % file_data[4])
    geofile.write("corner_lat: %f" % file_data[5])
    geofile.write("corner_lon: %f" % file_data[6])
    geofile.write("post_lat: %f" % file_data[7])
    geofile.write("post_lon: %f" % file_data[8])
    geofile.close()
    
    print ("auto_tree_height_many finished at " + (time.strftime("H:%M:%S")))

In [ ]:
def auto_tree_height_single_ISCE(directory, date1, date2, numLooks, noiselevel, flag_grad):
    # Extract ISCE parameters
    xmlfile = subprocess.getoutput('find ' + directory + 'int_' + date1 + '_' + date2 + '/ -name *Proc.xml')
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    root_tag = root.tag
    
    range_pixel_res = float(root.findall("./master/instr"))

In [ ]:
def read_rsc_data(filename, directory, param):
    # set default output value
    result = -1
    
    # Set filenmae for file to be searched
    rsc_file = dirextory + filenmae
    
    # Read parameters from file
    for line in open(rsc_file):
        if line.startswith(param):
            result = float(line.strip().split()[1])
    
    return result